In [2]:
#setup - rememeber to switch to tensorflow 2.3 kernel...
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import datetime
import trimesh
import time
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 2*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

print(tf.__version__)

%matplotlib notebook

%load_ext tensorboard

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180

2022-10-18 17:46:27.419613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 17:46:28.045486: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-18 17:46:28.914165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-18 17:46:28.914230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.0


2022-10-18 17:46:31.227558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 17:46:31.338774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 17:46:31.339038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Autosaving every 180 seconds


In [3]:
#load OFF file from ModelNet10 dir
start = time.time()
# fn = '/media/derm/06EF-127D1/ModelNet40/toilet/train/toilet_0069.off'
fn = '/media/derm/06EF-127D1/ModelNet40/airplane/train/airplane_0124.off'

M = trimesh.load(fn)
#rescale
print(M.scale)
# M.convert_units('m', guess = True)
test = trimesh.sample.sample_surface(M, 1000)
test2 = trimesh.sample.sample_surface(M, 1000)[0]

test2 = test2/M.scale

# print(test[0])
print("took ", time.time() - start, "seconds")


2483.799317389394
took  0.854686975479126 seconds


In [4]:
#use Vedo to plot OG and subsampled surfaces
plt1 = Plotter(N = 1, axes = 7, bg = (1, 1, 1), interactive = True)
disp = []

# disp.append(Points(M.vertices, c = 'blue', r = 4))
disp.append(Points(test[0], c = 'red', r = 5))
toilet = Mesh(M).c("gray").alpha(0.2)
disp.append(toilet)

# disp.append(Points(x_train[50,:,:].numpy(), c = 'blue', r = 5)) #test drawing unknown point cloud

plt1.show(disp, "surface sampling test")
ViewInteractiveWidget(plt1.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1397)

In [4]:
#define rotation matrix used to transform point clouds
def R_tf(angs):
    if len(tf.shape(angs)) == 1:
        angs = angs[None,:]
    phi = angs[:,0]
    theta = angs[:,1]
    psi = angs[:,2]
    mat = tf.Variable([[cos(theta)*cos(psi), sin(psi)*cos(phi) + sin(phi)*sin(theta)*cos(psi), sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi)],
                       [-sin(psi)*cos(theta), cos(phi)*cos(psi) - sin(phi)*sin(theta)*sin(psi), sin(phi)*cos(psi) + sin(theta)*sin(psi)*cos(phi)],
                       [sin(theta), -sin(phi)*cos(theta), cos(phi)*cos(theta)]
                        ])
    mat = tf.transpose(mat, [2, 0, 1])
    mat = tf.squeeze(mat)
    return mat

# determine euler angles from rotation matrix
def R2Euler(mat):
    if len( tf.shape(mat) ) == 2:
        mat = mat[None, :, :]
    R_sum = np.sqrt(( mat[:,0,0]**2 + mat[:,0,1]**2 + mat[:,1,2]**2 + mat[:,2,2]**2 ) / 2)
    phi = np.arctan2(-mat[:,1,2],mat[:,2,2])
    theta = np.arctan2(mat[:,0,2], R_sum)
    psi = np.arctan2(-mat[:,0,1], mat[:,0,0])
    angs = np.array([phi, theta, psi])
    return angs

In [5]:
#generate toy dataset using files in the ModelNet40 repository
import os

numMeshes = 40000 #300 #344
ptsPerCloud = 100 #was 25 in OG method 
iterPerMesh = 10 #number of times to sample clouds from each mesh

#init vector to store sampled point clouds
x = np.zeros([numMeshes*iterPerMesh, ptsPerCloud*2, 3])
#init vector to store transformations 
# y = np.zeros([numMeshes*iterPerMesh, 6]) #rotation and translation
y = np.zeros([numMeshes*iterPerMesh, 3]) #if only considering translations

#scale trans and rotation params so outputs are equally weighted
trans_scale = 1.0
rot_scale = 0.2 #0.2

for i in range(numMeshes):
#     if i % 5 == 0:
#         print(i)
    
    #randomly choose object type
    obj_categorty = os.listdir('/media/derm/06EF-127D1/ModelNet40/')[int(40*np.random.rand())]
    obj_id_list = os.listdir('/media/derm/06EF-127D1/ModelNet40/' + obj_categorty + '/train/')
#     obj_id = obj_id_list[int(len(obj_id_list)*np.random.rand())] #full objects
    obj_id = obj_id_list[int(10*np.random.rand())] #debug - just look at a few per category
    print("iter:",i, obj_id)
    fn = '/media/derm/06EF-127D1/ModelNet40/' + obj_categorty + '/train/' + obj_id
    
#     fn = '/media/derm/06EF-127D1/ModelNet40/toilet/train/toilet_%04d.off' %(i+1) #loop through toilets
#     fn = '/media/derm/06EF-127D1/ModelNet40/bed/train/bed_0059.off'
    M = trimesh.load(fn)

    #more efficient to sample all points at once and then just use some for each frame
    sam1 = trimesh.sample.sample_surface(M, iterPerMesh*ptsPerCloud)[0] #get keyframe scan
    sam2 = trimesh.sample.sample_surface(M, iterPerMesh*ptsPerCloud)[0] #get new scan
#     sam2 = sam1 + 0.01*np.random.randn(np.shape(sam1)[0], 3) #copy point locations and add some noise
    
    sam1 = sam1/M.scale
    sam2 = sam2/M.scale
    
    for j in range(iterPerMesh):
        #rotate keyframe
        angs1 = 0.0*tf.random.normal([3])
        rot1 = R_tf(angs1)
        #rotate scan 2 relative to keyframe
        angs2 = rot_scale*tf.random.normal([3])
#         rot2 = R_tf(angs1 + angs2) #was this (wrong??)
        angs2 = tf.zeros([3]) # ~~~~~~~~~~~~~~~ zero out rotation (for debug) ~~~~~~~~~~~~~~~~~~~~~~
#         rot2 = tf.matmul(R_tf(angs1), R_tf(angs2)) #was this (wrong??)
        rot2 = tf.matmul(R_tf(angs2), R_tf(angs1)) #test
        
        # randomly grow/shrink each point cloud before translation
        scale = 1. + 0.2*tf.random.normal([1])[0]

        x[i*iterPerMesh + j, :ptsPerCloud, :] = sam1[j*ptsPerCloud:(j+1)*ptsPerCloud].dot(rot1.numpy())*scale           
            
        trans = trans_scale*tf.random.normal([3])
        #was this (incorrect for large angle deviation?)
#         sam2_j = trans + sam2[j*ptsPerCloud:(j+1)*ptsPerCloud].dot(rot1.numpy()).dot(rot2.numpy())*scale 
        sam2_j = trans + sam2[j*ptsPerCloud:(j+1)*ptsPerCloud].dot(rot2.numpy())*scale 
        x[i*iterPerMesh + j, ptsPerCloud:, :] = sam2_j

        #save transformation as y
        y[i*iterPerMesh + j,:3] = trans.numpy()#/trans_scale
#         y[i*iterPerMesh + j,3:] = angs2.numpy()#/rot_scale

iter: 0 night_stand_0004.off


2022-10-05 23:12:02.786032: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 23:12:02.786663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-05 23:12:02.786823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-05 23:12:02.786940: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

iter: 1 cone_0008.off
iter: 2 wardrobe_0009.off
iter: 3 toilet_0002.off
iter: 4 car_0005.off
iter: 5 dresser_0001.off
iter: 6 car_0001.off
iter: 7 xbox_0004.off
iter: 8 chair_0002.off
iter: 9 guitar_0007.off
iter: 10 xbox_0002.off
iter: 11 guitar_0005.off
iter: 12 cone_0003.off
iter: 13 monitor_0009.off
iter: 14 laptop_0004.off
iter: 15 airplane_0003.off
iter: 16 car_0003.off
iter: 17 bathtub_0022.off
iter: 18 bench_0007.off
iter: 19 door_0006.off
iter: 20 vase_0002.off
iter: 21 lamp_0009.off
iter: 22 laptop_0010.off
iter: 23 stool_0007.off
iter: 24 car_0010.off
iter: 25 tent_0006.off
iter: 26 chair_0002.off
iter: 27 stool_0008.off
iter: 28 plant_0008.off
iter: 29 bottle_0007.off
iter: 30 bottle_0003.off
iter: 31 mantel_0002.off
iter: 32 guitar_0004.off
iter: 33 stairs_0007.off
iter: 34 sofa_0003.off
iter: 35 bowl_0003.off
iter: 36 table_0006.off
iter: 37 stairs_0003.off
iter: 38 table_0004.off
iter: 39 night_stand_0003.off
iter: 40 tent_0006.off
iter: 41 car_0005.off
iter: 42 monitor_

iter: 323 stairs_0004.off
iter: 324 table_0002.off
iter: 325 bed_0010.off
iter: 326 cup_0008.off
iter: 327 night_stand_0010.off
iter: 328 bed_0010.off
iter: 329 radio_0006.off
iter: 330 keyboard_0009.off
iter: 331 piano_0007.off
iter: 332 sofa_0009.off
iter: 333 keyboard_0005.off
iter: 334 dresser_0005.off
iter: 335 desk_0001.off
iter: 336 toilet_0003.off
iter: 337 tv_stand_0009.off
iter: 338 monitor_0002.off
iter: 339 stool_0004.off
iter: 340 laptop_0001.off
iter: 341 bed_0005.off
iter: 342 bottle_0004.off
iter: 343 person_0007.off
iter: 344 guitar_0010.off
iter: 345 bed_0001.off
iter: 346 cup_0004.off
iter: 347 laptop_0007.off
iter: 348 bookshelf_0005.off
iter: 349 bowl_0008.off
iter: 350 airplane_0005.off
iter: 351 stool_0002.off
iter: 352 vase_0009.off
iter: 353 piano_0010.off
iter: 354 laptop_0007.off
iter: 355 stool_0001.off
iter: 356 table_0010.off
iter: 357 lamp_0002.off
iter: 358 tent_0005.off
iter: 359 guitar_0009.off
iter: 360 keyboard_0005.off
iter: 361 toilet_0010.off
iter

iter: 642 tv_stand_0003.off
iter: 643 laptop_0008.off
iter: 644 lamp_0009.off
iter: 645 plant_0005.off
iter: 646 sink_0010.off
iter: 647 vase_0004.off
iter: 648 plant_0007.off
iter: 649 airplane_0002.off
iter: 650 door_0006.off
iter: 651 glass_box_0006.off
iter: 652 radio_0007.off
iter: 653 flower_pot_0006.off
iter: 654 cone_0005.off
iter: 655 xbox_0002.off
iter: 656 cone_0006.off
iter: 657 bottle_0003.off
iter: 658 tv_stand_0003.off
iter: 659 piano_0010.off
iter: 660 bench_0001.off
iter: 661 bed_0010.off
iter: 662 airplane_0001.off
iter: 663 guitar_0005.off
iter: 664 door_0003.off
iter: 665 mantel_0004.off
iter: 666 guitar_0006.off
iter: 667 dresser_0006.off
iter: 668 person_0006.off
iter: 669 night_stand_0006.off
iter: 670 xbox_0008.off
iter: 671 sink_0001.off
iter: 672 xbox_0002.off
iter: 673 cone_0007.off
iter: 674 radio_0003.off
iter: 675 tent_0003.off
iter: 676 cup_0008.off
iter: 677 sofa_0004.off
iter: 678 sofa_0004.off
iter: 679 glass_box_0001.off
iter: 680 sink_0007.off
iter: 

...

iter: 38792 airplane_0002.off
iter: 38793 chair_0004.off
iter: 38794 curtain_0001.off
iter: 38795 range_hood_0004.off
iter: 38796 monitor_0002.off
iter: 38797 bathtub_0005.off
iter: 38798 bench_0003.off
iter: 38799 tent_0008.off
iter: 38800 tv_stand_0003.off
iter: 38801 stool_0006.off
iter: 38802 dresser_0002.off
iter: 38803 lamp_0009.off
iter: 38804 night_stand_0001.off
iter: 38805 xbox_0003.off
iter: 38806 lamp_0007.off
iter: 38807 airplane_0004.off
iter: 38808 bed_0009.off
iter: 38809 bookshelf_0008.off
iter: 38810 lamp_0008.off
iter: 38811 stairs_0008.off
iter: 38812 person_0004.off
iter: 38813 toilet_0007.off
iter: 38814 range_hood_0008.off
iter: 38815 monitor_0004.off
iter: 38816 mantel_0008.off
iter: 38817 bowl_0003.off
iter: 38818 sofa_0001.off
iter: 38819 desk_0007.off
iter: 38820 sofa_0004.off
iter: 38821 laptop_0003.off
iter: 38822 door_0001.off
iter: 38823 sink_0005.off
iter: 38824 sofa_0001.off
iter: 38825 keyboard_0006.off
iter: 38826 stool_0001.off
iter: 38827 bathtub_00

iter: 39089 desk_0008.off
iter: 39090 range_hood_0002.off
iter: 39091 bottle_0010.off
iter: 39092 guitar_0006.off
iter: 39093 bookshelf_0005.off
iter: 39094 dresser_0001.off
iter: 39095 stool_0009.off
iter: 39096 night_stand_0005.off
iter: 39097 tent_0008.off
iter: 39098 desk_0009.off
iter: 39099 bed_0001.off
iter: 39100 chair_0005.off
iter: 39101 monitor_0002.off
iter: 39102 radio_0003.off
iter: 39103 monitor_0002.off
iter: 39104 door_0007.off
iter: 39105 tv_stand_0001.off
iter: 39106 keyboard_0003.off
iter: 39107 vase_0009.off
iter: 39108 sofa_0004.off
iter: 39109 vase_0002.off
iter: 39110 stairs_0004.off
iter: 39111 plant_0004.off
iter: 39112 plant_0007.off
iter: 39113 laptop_0001.off
iter: 39114 range_hood_0009.off
iter: 39115 laptop_0006.off
iter: 39116 person_0006.off
iter: 39117 car_0002.off
iter: 39118 airplane_0006.off
iter: 39119 mantel_0004.off
iter: 39120 person_0010.off
iter: 39121 curtain_0009.off
iter: 39122 bowl_0002.off
iter: 39123 table_0004.off
iter: 39124 bowl_0008.

iter: 39384 sofa_0006.off
iter: 39385 xbox_0009.off
iter: 39386 piano_0005.off
iter: 39387 lamp_0003.off
iter: 39388 glass_box_0003.off
iter: 39389 airplane_0010.off
iter: 39390 tv_stand_0001.off
iter: 39391 keyboard_0006.off
iter: 39392 dresser_0004.off
iter: 39393 toilet_0006.off
iter: 39394 curtain_0003.off
iter: 39395 flower_pot_0007.off
iter: 39396 mantel_0005.off
iter: 39397 sink_0008.off
iter: 39398 xbox_0003.off
iter: 39399 chair_0009.off
iter: 39400 table_0002.off
iter: 39401 vase_0010.off
iter: 39402 range_hood_0002.off
iter: 39403 person_0006.off
iter: 39404 tent_0004.off
iter: 39405 bench_0003.off
iter: 39406 tv_stand_0009.off
iter: 39407 dresser_0008.off
iter: 39408 chair_0001.off
iter: 39409 curtain_0002.off
iter: 39410 keyboard_0001.off
iter: 39411 bench_0005.off
iter: 39412 sink_0002.off
iter: 39413 chair_0004.off
iter: 39414 sink_0002.off
iter: 39415 curtain_0010.off
iter: 39416 stool_0010.off
iter: 39417 vase_0009.off
iter: 39418 stool_0004.off
iter: 39419 airplane_00

iter: 39678 glass_box_0007.off
iter: 39679 chair_0003.off
iter: 39680 xbox_0004.off
iter: 39681 table_0007.off
iter: 39682 dresser_0002.off
iter: 39683 xbox_0001.off
iter: 39684 night_stand_0003.off
iter: 39685 tv_stand_0008.off
iter: 39686 toilet_0001.off
iter: 39687 chair_0009.off
iter: 39688 piano_0007.off
iter: 39689 bench_0010.off
iter: 39690 xbox_0006.off
iter: 39691 lamp_0003.off
iter: 39692 cone_0009.off
iter: 39693 sink_0006.off
iter: 39694 car_0009.off
iter: 39695 wardrobe_0004.off
iter: 39696 piano_0005.off
iter: 39697 monitor_0007.off
iter: 39698 cone_0001.off
iter: 39699 door_0009.off
iter: 39700 vase_0002.off
iter: 39701 tv_stand_0009.off
iter: 39702 cone_0010.off
iter: 39703 curtain_0010.off
iter: 39704 range_hood_0002.off
iter: 39705 tv_stand_0001.off
iter: 39706 piano_0006.off
iter: 39707 lamp_0007.off
iter: 39708 plant_0004.off
iter: 39709 laptop_0004.off
iter: 39710 toilet_0009.off
iter: 39711 bathtub_0021.off
iter: 39712 person_0007.off
iter: 39713 tent_0004.off
ite

iter: 39975 tv_stand_0008.off
iter: 39976 lamp_0006.off
iter: 39977 vase_0009.off
iter: 39978 desk_0008.off
iter: 39979 keyboard_0010.off
iter: 39980 laptop_0006.off
iter: 39981 chair_0006.off
iter: 39982 lamp_0004.off
iter: 39983 xbox_0007.off
iter: 39984 wardrobe_0006.off
iter: 39985 range_hood_0008.off
iter: 39986 curtain_0009.off
iter: 39987 flower_pot_0008.off
iter: 39988 tv_stand_0009.off
iter: 39989 flower_pot_0004.off
iter: 39990 bench_0009.off
iter: 39991 desk_0001.off
iter: 39992 laptop_0006.off
iter: 39993 xbox_0002.off
iter: 39994 tent_0006.off
iter: 39995 table_0009.off
iter: 39996 cone_0007.off
iter: 39997 guitar_0002.off
iter: 39998 cup_0006.off
iter: 39999 car_0010.off


In [ ]:
#TODO: generate dataset using all models from ModelNet40 dataset

In [6]:
#split into train and test sets, save to file
split = 0.9
x_train = x[:int(split*np.shape(x)[0])]
np.save('/media/derm/06EF-127D1/TrainingData/ModelNet40/full_simple_ModelNet40_x_train', x_train)
x_test = x[int(split*np.shape(x)[0]):]
np.save('/media/derm/06EF-127D1/TrainingData/ModelNet40/full_simple_ModelNet40_x_test', x_test)
y_train = y[:int(split*np.shape(y)[0])]
np.save('/media/derm/06EF-127D1/TrainingData/ModelNet40/full_simple_ModelNet40_y_train', y_train)
y_test = y[int(split*np.shape(y)[0]):]
np.save('/media/derm/06EF-127D1/TrainingData/ModelNet40/full_simple_ModelNet40_y_test', y_test)

print(np.shape(x_train))
print(np.shape(y_train))

(360000, 200, 3)
(360000, 3)


In [ ]:
#load data from memory
numMeshes = 300 #344
ptsPerCloud = 100 #50 #was 25 in OG method 
iterPerMesh = 50 #100  #number of times to sample clouds from each mesh
trans_scale = 10.0 #2.0
rot_scale = 0.2 #0.2

x_train = np.load('/media/derm/06EF-127D1/TrainingData/ModelNet40/full_ModelNet40_x_train.npy')
y_train = np.load('/media/derm/06EF-127D1/TrainingData/ModelNet40/full_ModelNet40_y_train.npy')
x_test = np.load('/media/derm/06EF-127D1/TrainingData/ModelNet40/full_ModelNet40_x_test.npy')
y_test = np.load('/media/derm/06EF-127D1/TrainingData/ModelNet40/full_ModelNet40_y_test.npy')

# #limit ptsPerCloud of loaded data -----------------------------------
# halflen = np.shape(x_train)[1]//2
# x_train = np.concatenate((x_train[:,:ptsPerCloud,:], x_train[:,halflen:(halflen + ptsPerCloud),:]), axis = 1)
# halflen = np.shape(x_test)[1]//2
# x_test = np.concatenate((x_test[:,:ptsPerCloud,:], x_test[:,halflen:(halflen + ptsPerCloud),:]), axis = 1)
# print(np.shape(x_train)) #starts out at 256 per cloud
# print(np.shape(y_train)) 
# print(np.shape(x_test))
# print(np.shape(y_test))
# #---------------------------------------------------------------------


### Load Lidar Data (Alternate Option) 

In [ ]:
#Train on data generated from KITTI dataset
#(from drive 005 snippet)
# d1 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_KITTI_scan1_50_shifted.txt")
# d2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_KITTI_scan2_50_shifted.txt")
# gt = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_KITTI_ground_truth_50_shifted.txt")

# #full urban drive (much larger)
# d1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_KITTI_FULL_scan1_to400.npy")
# d2 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_KITTI_FULL_scan2_to400.npy")
# gt = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_KITTI_FULL_ground_truth_to400.npy")

# toy data set generated in MatLab
d1 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/scan1_1k.txt")
d2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/scan2_1k.txt")
gt = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ground_truth_1k.txt")

points_per_sample = 50  #num pts per scan - defined in MatLab script
scan1 = tf.reshape(tf.convert_to_tensor(d1), [-1, points_per_sample, 3])
scan2 = tf.reshape(tf.convert_to_tensor(d2), [-1, points_per_sample, 3])
gt = tf.convert_to_tensor(gt)

#split data into training and validation sets
tsplit = 0.95 #this fraction goes into training
ntrain = int(tsplit*tf.shape(scan1)[0].numpy())
x_train = tf.concat((scan1[:ntrain], scan2[:ntrain]), axis = 1)
x_test = tf.concat((scan1[ntrain:], scan2[ntrain:]), axis = 1)
y_train = gt[:ntrain]
y_test = gt[ntrain:]

print(tf.shape(y_train))
y_train = tf.concat((y_train, tf.zeros(tf.shape(y_train), dtype = tf.double)), axis = 1)
# y_train = tf.concat((y_train, tf.zeros(tf.shape(y_train), dtype = tf.float32)), axis = 1)
print(tf.shape(y_train))
print(tf.shape(x_train))

In [ ]:
x_train = tf.convert_to_tensor(x_train)
y_train = tf.convert_to_tensor(y_train)
x_test = tf.convert_to_tensor(x_test)
y_test = tf.convert_to_tensor(y_test)
print(tf.shape(x_train))
print(tf.shape(y_train))

#TEST
#just translation case ------------
# y_train = y_train[:,:3]
# print(tf.shape(y_train))
#----------------------------------

## Create TF dataset to augment data

In [ ]:
def augment(points, gt):
    print(np.shape(points))
    print(np.shape(gt))

    
    # jitter points
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)

    # shuffle points in first point cloud
    points = tf.concat([tf.random.shuffle(points[:100]), tf.random.shuffle(points[100:])], axis = 0)

#     #TEST- shuffle and randomly translate cloud 2 and ground truth
#     shift_scale = 0.25 #don't want to make this too large for PC to reach in 1 iter(?)
#     shift = tf.cast(tf.concat([shift_scale*tf.random.normal([3]), tf.zeros(3)], axis=0) , tf.float64)
#     points = tf.concat([tf.random.shuffle(points[:100]) + shift[:3], tf.random.shuffle(points[100:])], axis = 0)
#     gt += shift
    
    return points, gt

BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(len(x_train)).map(augment).batch(BATCH_SIZE)
print(train_dataset)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.shuffle(len(x_test)).map(augment).batch(BATCH_SIZE)
print(val_dataset)

# Train Network

| MAE      | Batch Size     |    Epochs | Train Dataset | Notes | Network|
| ----------- | ----------- | --------- | --------   | | |
| 0.055  | 64 |  30   | toilet 69, 5k samles | 50 pts per scan    | Net |
| 0.067  | 16 |  30   | toilet 69, 1800 samles | 512 pts per scan    | Net |
| 0.11  | 64 |  30   | 20 toilets, 100 PCs each |no augmentation between epochs| Net |
| 0.065  | 64 |  30   | 20 toilets, 100 PCs each |Using tf.data.map(augment)| Net |

In [ ]:
from network import Net #mine
# from network import PointNet as Net #uses builtin PointNetConv2Layer
# from network import PCRnet as Net #PCR-Net baseline
np.random.seed(1337)
runLen =  32

def scheduler(epoch, learning_rate):
    part1 = runLen//4
    part2 = 2*runLen//4
    part3 = 3*runLen//4
    
    if epoch < part1:
        learning_rate = 0.001
        return learning_rate
    if epoch >= part1 and epoch < part2:
        learning_rate = 0.0001       
        return learning_rate
    if epoch >= part2 and epoch < part3:
        learning_rate = 0.00005     
        return learning_rate
    if epoch >= part3:
        learning_rate = 0.00001
        return learning_rate

model = Net()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = tf.keras.losses.MeanAbsoluteError()) #was MeanAbsoluteError()

summary = model.summary()
print(summary)
scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
cp = tf.keras.callbacks.ModelCheckpoint("CP.kmod", monitor = 'val_loss', save_best_only = True) 

log_dir = "runs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#was this
# trace = model.fit(x = x_train, y = y_train, batch_size = 128, epochs=runLen, verbose=1, 
#                   validation_split = 0.1, shuffle=True, callbacks = [cp,scheduler])

# #using TF.dataset.augmen
trace = model.fit(train_dataset, epochs=runLen, validation_data = val_dataset, 
                  verbose=1, callbacks = [cp,scheduler])

In [ ]:
# Network Loss Plots
from matplotlib import pyplot as plt
fig0, ax0 = plt.subplots()
ax0.plot(trace.history['loss'], '-')
ax0.plot(trace.history['val_loss'], '-')
ax0.legend(['train', 'val'], loc='upper left')
ax0.set_xlabel('iteration')
ax0.set_ylabel('Mean Absolute Error')

In [ ]:
#temp- get center location of each point cloud to make sure net isn't just matching centroids
idx = int(np.floor(100*np.random.randn()))
mu1 = tf.math.reduce_mean(x_train[idx, :ptsPerCloud, :], axis = 0)
mu2 = tf.math.reduce_mean(x_train[idx, ptsPerCloud:, :], axis = 0) - (y_train[idx,:3]*trans_scale)
print(mu1)
print(mu2)
center_error = mu1 - mu2
print(center_error)

In [ ]:
#look at errors at never-before-seen test data generated from similar objects in ModelNet10
guess = model.predict(x_train[:4])
error = y_train[:4] - guess
print(guess)
print(y_train[:4])
print(error)

In [7]:
#Generate special test data for this visualization (evenly sampled)
#  (doing this so we can draw the underlying model from which points were sampled)
fn = '/media/derm/06EF-127D1/ModelNet40/toilet/train/toilet_0310.off' #0310 looks best
# fn = 'C:/Users/Derm/Desktop/big/ModelNet10/toilet/train/toilet_0069.off' #simple shape
# fn = 'C:/Users/Derm/Desktop/big/ModelNet10/bed/train/bed_0250.off' 
# fn = 'C:/Users/Derm/Desktop/big/ModelNet10/sofa/train/sofa_0250.off' 

M = trimesh.load(fn)

n_tests = 1 #number of test samples to generate
#init vector to store sampled point clouds
x_test2 = np.zeros([n_tests, ptsPerCloud*2, 3])
#init vector to store transformations 
y_test2 = np.zeros([n_tests, 6]) #rotation and translation

sam1 = trimesh.sample.sample_surface(M, n_tests*ptsPerCloud)[0] #get keyframe scan
sam2 = trimesh.sample.sample_surface(M, n_tests*ptsPerCloud)[0] #get new scan

for j in range(n_tests):
    angs1 = 0.5*tf.random.normal([3])    #rotate keyframe
    rot1 = R_tf(angs1)
    angs2 = rot_scale*tf.random.normal([3])     #rotate scan 2 relative to keyframe
    angs2 = tf.zeros([3]) # ~~~~~~~~~~~~~~~ zero out rotation (for debug) ~~~~~~~~~~~~~~~~~~~~~~
    rot2 = R_tf(angs2)
    #     rot_combined = R_tf(angs1 + angs2) #was this
    rot_combined = tf.matmul(R_tf(angs1), R_tf(angs2))
    
    x_test2[j, :ptsPerCloud, :] = sam1[j*ptsPerCloud:(j+1)*ptsPerCloud].dot(rot1.numpy())         

    trans = trans_scale*tf.random.normal([3])
    #was this
    sam2_j = trans + sam2[j*ptsPerCloud:(j+1)*ptsPerCloud].dot(rot_combined.numpy()) #transform scan
    #DEBUG
#     sam2_j = (sam2[j*ptsPerCloud:(j+1)*ptsPerCloud]+trans.numpy()).dot(rot_combined.numpy()) #transform scan
    x_test2[j, ptsPerCloud:, :] = sam2_j

    #save transformation as y
    y_test2[j,:3] = trans.numpy()/trans_scale
    y_test2[j,3:] = angs2.numpy()/rot_scale
print(tf.shape(x_test2))

tf.Tensor([  1 200   3], shape=(3,), dtype=int32)


In [8]:
#visualize network performance on evenly sampled data
t = 0 #test number to draw
niter = 5 #number of iterations to run network for

plt2 = Plotter(N = 3, axes = 4, bg = (1, 1, 1), interactive = True)
disp1 = [] #before estimated transformation (drawn on left)
disp2 = [] #after 1 transformation (drawn in center)
disp3 = [] #after niter transformations

#draw first viz (untransformed set of scans)-------------------
scan1 = Mesh(M).c("red").alpha(0.1)#.rotate(90, axis = (0,0,1))
scan1.applyTransform(rot1.numpy().T)
disp1.append(scan1)
disp1.append(Points(x_test2[0,:ptsPerCloud], c = 'red', r = 5))

scan2 = Mesh(M).c("blue").alpha(0.1)
scan2.applyTransform(rot_combined.numpy().T)
# scan2.pos(y_test2[t,0], y_test2[t,1], y_test2[t,2])
scan2.pos(y_test2[t,0]*trans_scale, y_test2[t,1]*trans_scale, y_test2[t,2]*trans_scale)
disp1.append(scan2)
disp1.append(Points(x_test2[0,ptsPerCloud:], c = 'blue', r = 5))

#FOR DEBUG - draw ground truth transformation in green so I can be sure which order is correct
# correct = (x_test2[0,ptsPerCloud:] - y_test2[0,:3]*trans_scale).dot(R_tf(y_test2[0,3:]*rot_scale).numpy().T)
# temp = Points(correct, c = 'green', r = 5)
# disp1.append(temp)
#---------------------------------------------------------------

#draw esatimated soln after 1 iteration ------------------------
ans_cum = model.predict(x_test2)[t]
ans_cum[:3] = ans_cum[:3]*trans_scale
ans_cum[3:] = ans_cum[3:]*rot_scale

#draw meshes
soln_est_rot = R_tf(ans_cum[3:])
scan2_transformed = Mesh(M).c("blue").alpha(0.1)
scan2_transformed.applyTransform(soln_est_rot.numpy().dot(rot_combined.numpy().T))
scan2_transformed.pos(y_test2[t,0]*trans_scale - ans_cum[0], 
                      y_test2[t,1]*trans_scale - ans_cum[1], 
                      y_test2[t,2]*trans_scale - ans_cum[2])
disp2.append(scan2_transformed)
disp2.append(Mesh(M).c("red").alpha(0.1).applyTransform(rot1.numpy().T)) #draw keyframe

#add points
scan2_pts_transformed = (x_test2[0,ptsPerCloud:] - ans_cum[:3]).dot(soln_est_rot.numpy().T)
# scan2_pts_transformed = (x_test2[0,ptsPerCloud:]).dot(soln_est_rot.numpy().T) - ans_cum[:3]
disp2.append(Points(scan2_pts_transformed, c = 'blue', r = 5))

disp2.append(Points(x_test2[0,:ptsPerCloud], c = 'red', r = 5))
gt = y_test2[t].copy()
gt[:3] = gt[:3]*trans_scale
gt[3:] = gt[3:]*rot_scale
print("\n ground truth:", gt)
print("\n estimate from DNN after 1 iteration:", ans_cum)
#-----------------------------------------------------------------

# draw estiamted soln after n interations-------------------------
#TODO: need to figure out more compat way of representing sequential 6DOF transforms
for i in range(niter):
    #replace initial scan2 with transformed pc2 as input to network
    inlayer = tf.concat([x_test2[0][:ptsPerCloud], scan2_pts_transformed], axis = 0)[None, :, :]
    ans_i = model.predict(inlayer)[0]
    ans_i[:3] = ans_i[:3]*trans_scale
    ans_i[3:] = ans_i[3:]*rot_scale
    
#     soln_est_rot = tf.matmul(R_tf(ans_i[3:]), soln_est_rot)
    soln_est_rot = R_tf(ans_i[3:]) #test
    ans_cum[:3] = ans_cum[:3] + ans_i[:3]
#     ans_cum[3:] = R2Euler(soln_est_rot)[:,0]
    ans_cum[3:] = R2Euler(tf.matmul(R_tf(ans_i[3:]), soln_est_rot))[:,0]
    scan2_pts_transformed = (x_test2[0,ptsPerCloud:] - ans_cum[:3]).dot(soln_est_rot.numpy().T)
    
# print("\n estimate from DNN after", niter, "iterations: ", ans_cum) 

scan2_transformed_again = Mesh(M).c("blue").alpha(0.1)
scan2_transformed_again.applyTransform(soln_est_rot.numpy().dot(rot_combined.numpy().T)) #test
scan2_transformed_again.pos(y_test2[t,0]*trans_scale - ans_cum[0], 
                            y_test2[t,1]*trans_scale - ans_cum[1], 
                            y_test2[t,2]*trans_scale - ans_cum[2])
disp3.append(scan2_transformed_again)
disp3.append(Points(scan2_pts_transformed, c = 'blue', r = 5))
disp3.append(Points(x_test2[0,:ptsPerCloud], c = 'red', r = 5))
disp3.append(Mesh(M).c("red").alpha(0.1).applyTransform(rot1.numpy().T)) #keyframe

print("------- \n Final Error:", gt - ans_cum)
# #---------------------------------------------------------------

    
plt2.show(disp1, "initial transformation", at = 0)
plt2.show(disp2, "after 1 iteration", at = 1)
plt2.show(disp3, "after 5 iterations", at = 2)
ViewInteractiveWidget(plt2.window)

NameError: name 'model' is not defined

In [ ]:
# print(np.mean(x_test2[:ptsPerCloud], axis = 1))

In [ ]:
# model.save("DermNet_ModelNet_benchmark.kmod") #256 pts per cloud, MAE =~ 0.3177
# model.save("DermNet_ModelNet_benchmark.h5") #allows viz with Netron
# model = tf.keras.models.load_model("DermNet_ModelNet_benchmark.kmod")

# model.save("DermNet_ModelNet_trans_only.kmod") #256 pts per cloud, MAE = 0.0308
# model = tf.keras.models.load_model("DermNet_ModelNet_trans_only.kmod")
model.save("CP.kmod") #256 pts per cloud, MAE = 0.071

In [ ]:
#Can't just add euler angles in 3D...
a = tf.constant([[1., 2., 3.]])
A = R_tf(a)
b = tf.constant([[0.3, 0.2, 0.1]])
B = R_tf(b)
print(tf.matmul(A, B))
print(R_tf(a + b))


In [ ]:
print(A.numpy().T)
print(np.linalg.pinv(A.numpy()))

In [ ]:
tf.matmul(R_tf(tf.constant([0.,0.,0.])), R_tf(a))